In [8]:
import os
import numpy as np
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Function to preprocess and load images from a folder
def load_images_from_folder(folder, target_size=(256, 256), grayscale=True):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE if grayscale else cv2.IMREAD_COLOR)
        if img is not None:
            img = cv2.resize(img, target_size)  # Resize image
            img = cv2.GaussianBlur(img, (5, 5), 0)  # Apply Gaussian blur to remove noise
            _, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)  # Thresholding
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # Find contours
            for contour in contours:
                area = cv2.contourArea(contour)
                if area > 1000:  # Adjust this threshold based on your image size and noise level
                    x, y, w, h = cv2.boundingRect(contour)
                    roi = img[y:y+h, x:x+w]  # Extract ROI
                    images.append(roi.flatten())
    return images

# Paths to train and test folders
train_folder_fracture = r'D:\modelRF\kaggle_bone_xray_dataset_train\train\fractured'
train_folder_no_fracture = r'D:\modelRF\kaggle_bone_xray_dataset_train\train\not fractured'
test_folder_fracture = r'D:\modelRF\kaggle_bone_xray_dataset_test\test\fractured'
test_folder_no_fracture = r'D:\modelRF\kaggle_bone_xray_dataset_test\test\not fractured'

# Load images from train folders and preprocess them
train_images_fracture = load_images_from_folder(train_folder_fracture)
train_images_no_fracture = load_images_from_folder(train_folder_no_fracture)

# Load images from test folders and preprocess them
test_images_fracture = load_images_from_folder(test_folder_fracture)
test_images_no_fracture = load_images_from_folder(test_folder_no_fracture)

# Create labels (1 for fracture, 0 for no fracture)
train_labels_fracture = np.ones(len(train_images_fracture))
train_labels_no_fracture = np.zeros(len(train_images_no_fracture))

# Concatenate features and labels
X_train = np.concatenate([train_images_fracture, train_images_no_fracture])
y_train = np.concatenate([train_labels_fracture, train_labels_no_fracture])

# Create labels for test images
test_labels_fracture = np.ones(len(test_images_fracture))
test_labels_no_fracture = np.zeros(len(test_images_no_fracture))

# Concatenate features and labels for test set
X_test = np.concatenate([test_images_fracture, test_images_no_fracture])
y_test = np.concatenate([test_labels_fracture, test_labels_no_fracture])

# Split train set into train and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),  # Dropout layer with dropout rate of 0.5
    Dense(64, activation='relu'),
    Dropout(0.5),  # Dropout layer with dropout rate of 0.5
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Evaluate the model on validation set
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

# Evaluate the model on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Sample prediction on a single test image
sample_image_index = 0
sample_image = X_test[sample_image_index].reshape(1, -1)  # Reshape to match model inputs
sample_label = y_test[sample_image_index]
sample_prediction = model.predict(sample_image)[0][0]
predicted_class = 1 if sample_prediction >= 0.5 else 0

print("Sample Image Label:", sample_label)
print("Sample Image Prediction:", predicted_class)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6753,) + inhomogeneous part.

In [11]:
# Function to load and preprocess a single image
def load_and_preprocess_single_image(image_path, target_size=(100, 100)):
    img = cv2.imread(image_path)
    if img is not None:
        img = cv2.resize(img, target_size)  # Resize image
        return img.flatten()  # Flatten image pixels as features
    else:
        print("Error: Unable to load image.")
        return None

# Path to the single image you want to test
single_image_path = r'D:\modelRF\kaggle_demf_frac\29-rotated3-rotated1-rotated3.jpg'

# Load and preprocess the single image
single_image_features = load_and_preprocess_single_image(single_image_path)

if single_image_features is not None:
    # Reshape the features to match model input
    single_image_features = single_image_features.reshape(1, -1)
    
    # Predict using the trained Random Forest model
    single_image_prediction = rf_classifier.predict(single_image_features)[0]
    
    # Print the prediction
    if single_image_prediction == 1:
        print("The image contains a fracture.")
    else:
        print("The image does not contain a fracture.")


The image contains a fracture.


In [4]:
# Function to load and preprocess a single image
def load_and_preprocess_single_image(image_path, target_size=(100, 100)):
    img = cv2.imread(image_path)
    if img is not None:
        img = cv2.resize(img, target_size)  # Resize image
        return img.flatten()  # Flatten image pixels as features
    else:
        print("Error: Unable to load image.")
        return None

# Path to the single image you want to test
single_image_path = r'D:\modelRF\kaggle_demf_unfrac\7-rotated1-rotated1.jpg'

# Load and preprocess the single image
single_image_features = load_and_preprocess_single_image(single_image_path)

if single_image_features is not None:
    # Reshape the features to match model input
    single_image_features = single_image_features.reshape(1, -1)
    
    # Predict using the trained Random Forest model
    single_image_prediction = rf_classifier.predict(single_image_features)[0]
    
    # Print the prediction
    if single_image_prediction == 1:
        print("The image contains a fracture.")
    else:
        print("The image does not contain a fracture.")


The image does not contain a fracture.
